In [1]:
import tensorflow as tf
from ccgnet import experiment as exp
from ccgnet.finetune import *
from ccgnet import layers
from ccgnet.layers import *
import numpy as np
import time
from sklearn.metrics import balanced_accuracy_score
from ccgnet.Dataset import Dataset, DataLoader
from Featurize.Coformer import Coformer
from Featurize.Cocrystal import Cocrystal

/export/home/jyy/anaconda3/envs/ccdc/lib/python3.7/site-packages/ccdc/__init__.py:384: UserWarning: 
Could not find the CSD-System installation.

For further help with installing and configuring data please visit
the support page at https://www.ccdc.cam.ac.uk/csds_install_help

Usage of the CSD Python API without the CSD Data installed is not supported.

  warnings.warn(CSDHOME_NOT_FOUND_MSG)


In [2]:
data = Dataset('./Samples/CC_Table.tab', mol_blocks_dir='./Samples/Mol_Blocks.dir')
data.make_graph_dataset(Desc=1, A_type='OnlyCovalentBond', hbond=0, pipi_stack=0, contact=0, make_dataframe=True)

/export/home/jyy/anaconda3/envs/ccdc/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
RDKit ERROR: [20:43:14] Can't kekulize mol.  Unkekulized atoms: 12 14 16 17 19
RDKit ERROR: 
RDKit ERROR: [20:43:14] Can't kekulize mol.  Unkekulized atoms: 2 4 6 7 9
RDKit ERROR: 
RDKit ERROR: [20:43:16] Can't kekulize mol.  Unkekulized atoms: 2 4 6 7 9
RDKit ERROR: 
RDKit ERROR: [20:43:18] Can't kekulize mol.  Unkekulized atoms: 2 4 5 7 9
RDKit ERROR: 
RDKit ERROR: [20:43:20] Can't kekulize mol.  Unkekulized atoms: 2 4 6 7 9
RDKit ERROR: 
RDKit ERROR: [20:43:22] Can't kekulize mol.  Unkekulized atoms: 3 5 7 9 11
RDKit ERROR: 
RDKit ERROR: [20:43:22] Can't kekulize mol.  Unkekulized atoms: 0 2 4 6 8
RDKit ERROR:

Elapsed Time: 93.83 s


In [3]:
nico = eval(open('./Test/test_samples/Nicotinamide_Test.list').read())
carb = eval(open('./Test/test_samples/Carbamazepine_Test.list').read())
indo = eval(open('./Test/test_samples/Indomethacin_Test.list').read())
para = eval(open('./Test/test_samples/Paracetamol_Test.list').read())
pyre = eval(open('./Test/test_samples/Pyrene_Test.list').read())
test = list(set(nico + carb + indo + para + pyre))

/export/home/jyy/anaconda3/envs/ccdc/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
from sklearn.model_selection import KFold

cv = np.array([i for i in data.dataframe if i not in test])
np.random.shuffle(cv)
fold_10 = {}
fold_10['test'] = test
kf = KFold(n_splits=10, shuffle=True, random_state=1000)
n = 0
for train_idx, test_idx in kf.split(cv):
    fold = 'fold-{}'.format(n)
    fold_10[fold] = {}
    fold_10[fold]['train'] = cv[train_idx]
    fold_10[fold]['valid'] = cv[test_idx]
    n += 1

/export/home/jyy/anaconda3/envs/ccdc/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
class CCGNet_block(object):
    def build_model(self, inputs, is_training, global_step):
        V = inputs[0]
        A = inputs[1]
        labels = inputs[2]
        mask = inputs[3]
        graph_size = inputs[4]
        tags = inputs[5]
        global_state = inputs[6]
        subgraph_size = inputs[7]
        global_state = tf.reshape(global_state, [-1, int(global_state.get_shape()[-1].value/2)])
        V, global_state = CCGBlock(V, A, global_state, subgraph_size, no_filters=128, mask=mask, num_updates=global_step, is_training=is_training)
        V, global_state = CCGBlock(V, A, global_state, subgraph_size, no_filters=64, mask=mask, num_updates=global_step, is_training=is_training)
        V, global_state = CCGBlock(V, A, global_state, subgraph_size, no_filters=64, mask=mask, num_updates=global_step, is_training=is_training)
        V, global_state = CCGBlock(V, A, global_state, subgraph_size, no_filters=32, mask=mask, num_updates=global_step, is_training=is_training)
        V, global_state = CCGBlock(V, A, global_state, subgraph_size, no_filters=16, mask=mask, num_updates=global_step, is_training=is_training)
        V = layers.multi_head_global_attention(V, graph_size, num_head=10)
        global_state = tf.reshape(global_state, [-1, 32])
        V = tf.concat([V, global_state], 1)
        V = layers.make_embedding_layer(V, 256, name='FC')
        V = layers.make_bn(V, is_training, mask=None, num_updates=global_step)
        V = tf.nn.relu(V)
        out = layers.make_embedding_layer(V, 2, name='final')
        prob = layers.make_softmax_layer(out, axis=1, name=None)
        return out, labels

/export/home/jyy/anaconda3/envs/ccdc/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
start = time.time()
snapshot_path = './snapshot/'
model_name = 'CCGNet_block'
dataset_name = 'CC_Dataset'
History = {}
History['test_acc'] = []
History['val_acc'] = []
for fold in ['fold-{}'.format(i) for i in range(10)]:
    print('\n################ {} ################'.format(fold))
    train_data, valid_data, test_data = data.split(train_samples=fold_10[fold]['train'], test_samples=fold_10[fold]['valid'], val=True, val_samples=fold_10['test'])
    tf.reset_default_graph()
    model = CCGNet_block()
    model = exp.Model(model, train_data, valid_data, with_test=True, test_data=test_data, snapshot_path=snapshot_path, use_subgraph=True,
                      model_name=model_name, dataset_name=dataset_name+'/time_{}'.format(fold[-1]))
    history = model.fit(num_epoch=100, save_info=True, save_att=True, silence=0, train_batch_size=256,
                        metric='acc')
    History['test_acc'].append(max(history['test_acc']))
    History['val_acc'].append(history['val_acc'])
print('test_mean_acc:{:.4f}(+-{:.4f})'.format(np.array(History['test_acc']).mean()*100,np.array(History['test_acc']).std()*100))
print('val_mean_acc:{:.4f}(+-{:.4f})'.format(np.array(History['val_acc']).mean()*100,np.array(History['val_acc']).std()*100))
end = time.time()
time_gap = end-start
h = time_gap//3600
h_m = time_gap%3600
m = h_m//60
s = h_m%60
print('{}h {}m {}s'.format(int(h),int(m),round(s,2)))


################ fold-0 ################


/export/home/jyy/anaconda3/envs/ccdc/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Creating loss function and summaries
Preparing training
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/export/home/jyy/anaconda3/envs/ccdc/lib/python3.7/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/export/home/jyy/anaconda3/envs/ccdc/lib/python3.7/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Starting summaries
## Epoch 0 ==> Train Loss:0.40351, Train Acc:80.69, Valid Loss:0.11422, Valid Acc:96.25, Elapsed Time:19.13 s
## Epoch 1 ==> Train Loss:0.10482, Train Acc:96.47, Valid Loss:0.08594, Valid Acc:97.54, Elapsed Time:16.02 s
## Epoch 2 ==> Train Loss:0.07361, Train Acc:97.75, Valid Loss:0.07602, Valid Acc:97.67, Elapsed Time:16.34 s
## Epoch 3 ==> Train Loss:0.05837, Train Acc:98.10, Valid Loss:0.07055, Valid Acc:97.15, Elapsed Time:14.98 s
## Epoch 4 ==> Train Loss:0.04287, Train Acc:98.41, Valid Loss:0.05751, Valid Acc:98.45, Elapsed Time:15.81 s
## Epoch 5 ==> Train Loss:0.05024, Train Acc:98.43, Valid Loss:0.04168, Valid Acc:98.58, Elapsed Time:16.23 s
## Epoch 6 ==> Train Loss:0.04346, Train Acc:98.54, Valid Loss:0.06716, Valid Acc:98.06, Elapsed Time:15.34 s
## Epoch 7 ==> Train Loss:0.04176, Train Acc:98.66, Valid Loss:0.04643, Valid Acc:98.45, Elapsed Time:15.18 s
## Epoch 8 ==> Train Loss:0.03079, Train Acc:98.94, Valid Loss:0.04748, Valid Acc:98.45, Elapsed Time

...

/export/home/jyy/anaconda3/envs/ccdc/lib/python3.7/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/export/home/jyy/anaconda3/envs/ccdc/lib/python3.7/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Starting summaries
## Epoch 0 ==> Train Loss:0.20486, Train Acc:92.68, Valid Loss:0.08112, Valid Acc:97.02, Elapsed Time:18.76 s
## Epoch 1 ==> Train Loss:0.08102, Train Acc:97.20, Valid Loss:0.06122, Valid Acc:97.80, Elapsed Time:15.98 s
## Epoch 2 ==> Train Loss:0.06369, Train Acc:97.68, Valid Loss:0.05207, Valid Acc:98.32, Elapsed Time:15.86 s
## Epoch 3 ==> Train Loss:0.04314, Train Acc:98.50, Valid Loss:0.03722, Valid Acc:98.96, Elapsed Time:15.71 s
## Epoch 4 ==> Train Loss:0.03985, Train Acc:98.61, Valid Loss:0.03599, Valid Acc:99.09, Elapsed Time:15.43 s
## Epoch 5 ==> Train Loss:0.03872, Train Acc:98.61, Valid Loss:0.04827, Valid Acc:98.70, Elapsed Time:15.14 s
## Epoch 6 ==> Train Loss:0.02254, Train Acc:99.18, Valid Loss:0.03278, Valid Acc:98.58, Elapsed Time:15.18 s
## Epoch 7 ==> Train Loss:0.01666, Train Acc:99.43, Valid Loss:0.02744, Valid Acc:99.35, Elapsed Time:15.84 s
## Epoch 8 ==> Train Loss:0.02187, Train Acc:99.29, Valid Loss:0.02878, Valid Acc:98.96, Elapsed Time

/export/home/jyy/anaconda3/envs/ccdc/lib/python3.7/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/export/home/jyy/anaconda3/envs/ccdc/lib/python3.7/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Starting summaries
## Epoch 0 ==> Train Loss:0.21651, Train Acc:92.56, Valid Loss:0.10312, Valid Acc:96.63, Elapsed Time:19.38 s
## Epoch 1 ==> Train Loss:0.09402, Train Acc:96.84, Valid Loss:0.06014, Valid Acc:97.54, Elapsed Time:16.22 s
## Epoch 2 ==> Train Loss:0.05974, Train Acc:98.02, Valid Loss:0.05053, Valid Acc:97.54, Elapsed Time:15.38 s
## Epoch 3 ==> Train Loss:0.04525, Train Acc:98.49, Valid Loss:0.04753, Valid Acc:98.32, Elapsed Time:16.06 s
## Epoch 4 ==> Train Loss:0.04292, Train Acc:98.55, Valid Loss:0.03851, Valid Acc:98.70, Elapsed Time:16.13 s
## Epoch 5 ==> Train Loss:0.03621, Train Acc:98.84, Valid Loss:0.03957, Valid Acc:99.09, Elapsed Time:15.95 s
## Epoch 6 ==> Train Loss:0.02369, Train Acc:99.25, Valid Loss:0.02695, Valid Acc:99.35, Elapsed Time:16.20 s
## Epoch 7 ==> Train Loss:0.02672, Train Acc:99.18, Valid Loss:0.03589, Valid Acc:98.45, Elapsed Time:15.55 s
## Epoch 8 ==> Train Loss:0.02279, Train Acc:99.32, Valid Loss:0.04025, Valid Acc:98.58, Elapsed Time

NameError: name 'start' is not defined

In [7]:
from Featurize.MetricsReport import model_metrics_report

model_metrics_report('{}/{}/'.format(snapshot_path, model_name))

/export/home/jyy/anaconda3/envs/ccdc/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Balanced Accuracy,Nacc,Pacc,Precision,Recall,TN_number in each fold,TP_number in each fold,Test Accuracy,Train Accuracy,f1_score
CC_Dataset,97.9122(±0.9563),95.9586(±1.8904),99.8658(±0.0800),99.3921(±0.3010),99.8658(±0.0800),"110,94,100,98,99,90,98,95,91,126","663,678,673,675,673,683,675,678,682,647",99.3531(±0.2892),99.6633(±0.1950),99.6282(±0.1676)


In [8]:
from parselog import ParseTestLog, ParseTestLogEnsemble
import glob

val_list = glob.glob('{}/{}/{}/*/*val*'.format(snapshot_path, model_name, dataset_name))
l = []
for i in val_list:
    l.append(ParseTestLog(i))

/export/home/jyy/anaconda3/envs/ccdc/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
ens = ParseTestLogEnsemble(l)
print('####### Mean ########')
ens.Reports
print('####### Bagging ########')
ens_bagging = ens.Bagging

####### Mean ########
Balanced Accuracy: 94.99(+-2.81)
Negative Accuracy: 90.19(+-5.58)
Positive Accuracy: 99.79(+-0.42)
Accuracy: 96.38(+-2.02)
Precision: 94.93(+-2.76)
Recall: 99.79(+-0.42)
F1 Score: 97.28(+-1.48)
####### Bagging ########
BACC: 98.11
TP: 97.96
Nacc: 96.23
Pacc: 100.00
Recall: 100.00
F1: 98.97


/export/home/jyy/anaconda3/envs/ccdc/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
from parselog import TestAccForEachMol

print('\n######## Nicotinamide  ########')
nico_bagging = TestAccForEachMol(nico, val_list, is_return=1)
print('\n######## Carbamazepine ########')
carb_bagging = TestAccForEachMol(carb, val_list, is_return=1)
print('\n######## Indomethacin  ########')
indo_bagging = TestAccForEachMol(indo, val_list, is_return=1)
print('\n########  Paracetamol  ########')
para_bagging = TestAccForEachMol(para, val_list, is_return=1)
print('\n########    Pyrene     ########')
pyre_bagging = TestAccForEachMol(pyre, val_list, is_return=1)

/export/home/jyy/anaconda3/envs/ccdc/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



######## Nicotinamide  ########
BACC: 97.14(±3.50)
PACC: 100.00(±0.00)
NACC: 94.29(±7.00)
######### Bagging ########
BACC: 100.00
PACC: 100.00
NACC: 100.00

######## Carbamazepine ########
BACC: 92.50(±7.81)
PACC: 100.00(±0.00)
NACC: 85.00(±15.61)
######### Bagging ########
BACC: 100.00
PACC: 100.00
NACC: 100.00

######## Indomethacin  ########
BACC: 91.47(±4.25)
PACC: 100.00(±0.00)
NACC: 82.94(±8.50)
######### Bagging ########
BACC: 94.12
PACC: 100.00
NACC: 88.24

########  Paracetamol  ########
BACC: 98.12(±4.00)
PACC: 100.00(±0.00)
NACC: 96.25(±8.00)
######### Bagging ########
BACC: 100.00
PACC: 100.00
NACC: 100.00

########    Pyrene     ########
BACC: 99.83(±0.34)
PACC: 99.66(±0.69)
NACC: 100.00(±0.00)
######### Bagging ########
BACC: 100.00
PACC: 100.00
NACC: 100.00
